In [38]:
import numpy as np
import scipy as sp
from scipy.optimize import * 

In [93]:
n = 50
x = np.random.normal(size=(50,1))
e = np.random.normal(size=(50,1))
y = 0.5 + x + e
y = (y>=0).astype(int)
bigX = np.hstack((np.ones((50,1)),x))
(n,r) = bigX.shape

In [103]:

#def bigF(val):
#    return 1/(1+np.exp(-val))
# One line definition
bigF = lambda x: 1/(1+np.exp(-x))
littleF = lambda x: np.exp(-x)*bigF(x)**2
def littleF(val):
    return np.exp(-val)*bigF(val)**2
def logL(b):
    return (y*np.log(bigF(bigX.dot(b)))+(1-y)*np.log(1-bigF(bigX.dot(b)))).sum()
def deriv(b):
    suma = np.zeros((r,1)) 
    for i in range(n):
        curr = bigX[i,].reshape((r,1))
        suma += (y[i,]-bigF(curr.T.dot(b)))*curr
    return suma.reshape((r,))
def hess(b):
    suma = np.zeros((r,r)) 
    for i in range(n):
        curr = bigX[i,].reshape((r,1))
        suma += -littleF(curr.T.dot(b))*curr.dot(curr.T)
    suma
    return suma

In [96]:
b = root(deriv,x0=np.zeros((r,)),jac=hess)

In [105]:
g = root(lambda x: -x**2 + 2,x0=1,jac=lambda x: -2*x)
g

    fjac: array([[-1.]])
     fun: array([  8.88178420e-16])
 message: 'The solution converged.'
    nfev: 7
    njev: 1
     qtf: array([ -8.93145558e-10])
       r: array([ 2.82842925])
  status: 1
 success: True
       x: array([ 1.41421356])

In [97]:
# Fitted beta hats
b.x

array([ 1.5320155 ,  3.22397916])

In [98]:
# Variance covariance matrix for beta
-hess(b.x)/n

array([[ 0.11808532, -0.04431977],
       [-0.04431977,  0.03794551]])

In [81]:
import statsmodels.api as sm
sm.GLM(y,bigX,family=sm.families.Binomial()).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   50
Model:                            GLM   Df Residuals:                       48
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -27.573
Date:                Thu, 12 Jul 2018   Deviance:                       55.147
Time:                        19:29:22   Pearson chi2:                     49.9
No. Iterations:                     3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7852      0.343      2.286      0.022       0.112       1.458
x1             1.5133      0.523      2.894      0.004       0.488       2.538
==============================================================================
"""

In [102]:
outp1 = np.zeros((1000,2))
outp2 = np.zeros((1000,2))
for isim in range(1000):
    x = np.random.normal(size=(50,1))
    e = np.random.normal(size=(50,1))
    y = 0.5 + x + e
    y = (y>=0).astype(int)
    bigX = np.hstack((np.ones((50,1)),x))
    b = root(deriv,x0=np.zeros((r,)),jac=hess)
    outp1[isim,:] = b.x
    outp2[isim,:] = np.sqrt(np.diagonal(-hess(b.x)/n))
print(outp1.mean(0))
print(outp2.mean(0))

[ 0.93100165  1.8875021 ]
[ 0.38362526  0.28194649]
